In [1]:
import numpy as np
import scipy as sp
from scipy import special
from scipy.interpolate import CubicSpline
import astropy
from astropy.cosmology import FlatLambdaCDM
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('text', usetex = True)
from numpy import save

## Computes parameter combinations and load k and P(k) arrays

In [2]:
# number of redshifts run by axionCAMB
numz = 301
zlist = np.linspace(0,4.5,numz)
# number of parameter values run for each parameter
num_values = 9
# minimum and maximum axion fraction values
fa_min,fa_max = [0.0001,0.2]
# fiducial parameter values from Planck 2015
param_values = [0.02233,0.1198,fa_min,67.37,0.9652]
# 1-sigma values for each parameter from Planck 2015
sigma1 = [0.00015,0.0012,0,0.54,0.0042]
# convert 1-sigma to 2-sigma to create our prior (flat)
sigma2 = [2*sigma1[i] for i in range(len(sigma1))]
# create range of axion fractions
fa = np.linspace(fa_min,fa_max,num_values)
# convert axion fractions to axion energy densities
omega_a = param_values[1]*fa
# adjust the dark matter energy density to accommodate axions
omega_c = param_values[1]-omega_a
# axion mass list
ma = np.logspace(-29,-22,11)

# array of all parameter combinations
param_array = []
for i in range(len(param_values)):
    dim = []
    if i == 2:
        dim.append(np.asarray(num_values*[param_values[0]]))
        dim.append(omega_c)
        dim.append(omega_a)
        dim.append(np.asarray(num_values*[param_values[3]]))
        dim.append(np.asarray(num_values*[param_values[4]]))
    else:
        for j in range(len(param_values)):
            if i == j:
                dim.append(np.linspace(param_values[i]-sigma2[i],param_values[i]+sigma2[i],num_values))
            else:
                dim.append(np.asarray(num_values*[param_values[j]]))
    param_array.append(dim)
    
# reshape parameter combination array
a = np.transpose(param_array,(0,2,1))

# total Omega_matter
matter = [[a[i][j][0]+a[i][j][1]+a[i][j][2] for j in range(num_values)] for i in range(len(param_values))]

# angular diameter distance
d_a = [[[FlatLambdaCDM(H0=a[i][j][3],Om0=matter[i][j]).angular_diameter_distance(z_i).value for j in range(num_values)] for i in range(len(param_values))] for z_i in zlist]
d_a = np.transpose(d_a,(1,2,0))

# Hubble constant
E = [[[FlatLambdaCDM(H0=a[i][j][3],Om0=matter[i][j]).efunc(z_i) for j in range(num_values)] for i in range(len(param_values))] for z_i in zlist]
H0_i = np.expand_dims([[a[i][j][3] for j in range(num_values)] for i in range(len(param_values))],axis = 0)
H_z = np.transpose(H0_i*E,(1,2,0))

In [4]:
#need to down grade numpy to version 1.16.2 to avoid loading errors
k = np.load('../Axion-main/k_der.npy')[:,:,:,::-1]
pk = np.load('../Axion-main/pk_der.npy')[:,:,:,::-1]
pk_dimless = (np.power(k,3)*pk)/(2*(np.pi**2))


print(np.shape(pk))
print(np.shape(pk_dimless))
print(np.shape(a))

(5, 1, 25, 301, 660)
(5, 1, 25, 301, 660)
(5, 9, 5)


In [4]:
# plt.loglog(k[2][0][0][0],pk_dimless[2][0][0][0],color = 'k',label = r'$\Lambda {\rm CDM}$')
# plt.loglog(k[2][10][4][0],pk_dimless[2][10][4][0],color = 'r',label = r'$m_a = 10^{-22}{\rm eV}, f_a = 10\%$')
# plt.loglog(k[2][3][4][0],pk_dimless[2][3][4][0],color = 'b',label = r'$m_a = 10^{-27}{\rm eV}, f_a = 10\%$')
# plt.loglog(k[2][3][8][0],pk_dimless[2][3][8][0],color = 'g',label = r'$m_a = 10^{-27}{\rm eV}, f_a = 20\%$')
# plt.loglog(k[2][0][0][0],pk_dimless[2][0][0][0],color = 'k')
# plt.legend(frameon = False,loc = 'lower right',fontsize = 14)
# plt.xlabel(r'$k (h/{\rm Mpc})$',size = 14)
# plt.ylabel(r'$\Delta^2(k)$',size = 14)
# plt.xlim(0.01,100)
# plt.ylim(0.001,50)
# plt.savefig('k_vs_Pk')
# plt.show()
# plt.close('all')

## Create $z$-bins and shot noise

In [5]:
def bins(zmin = 0.15, zmax = 3.5, sigmaz0 = 0.04817):
    z = zmin
    zlist = [z]
    while z < zmax:
        z += sigmaz0*(1+z)
        zlist.append(np.round(z,2))
    #zbounds = [zlist[0],zlist[5],zlist[11],zlist[17],zlist[23],zlist[29]]
    zbounds = [zlist[0],zlist[4],zlist[9],zlist[14],zlist[19],zlist[24],zlist[29]]
    zavg = [np.round((zbounds[i]+zbounds[i+1])/2,2) for i in range(len(zbounds)-1)]
    return zbounds,zavg

zbounds = bins()[0]
zavg = bins()[1]
print(len(zavg))

def n(z, alpha = 2, zstar = 0.5 , beta = 1, norm = 4*np.pi*((10800/np.pi)**2)*50):#1126.6):
    return norm*np.power(z,alpha)*np.exp(-np.power(z/zstar,beta))

def I(a,b,z):
    return 0.5*(sp.special.erf(14.1421*(z-a)/(1+z))-sp.special.erf(14.1421*(z-b)/(1+z)))

zlist = np.arange(0.,5.,0.01)
bias = np.reshape(np.asarray(1+0.84*zlist),(1,len(zlist)))
ni = [[n(z)*I(zbounds[i],zbounds[i+1],z)/I(0,5,z) for z in zlist] for i in range(len(zbounds)-1)]
nibar = [np.trapz(ni[i],zlist,dx = 0.01) for i in range(len(ni))]
window = bias*np.asarray(ni)/np.reshape(np.asarray(nibar),(len(nibar),1))

# interpolate to match redshifts of axionCAMB and then export
zmin_camb = 0
zmax_camb = 4.5
numz_camb = 301
zlist_camb = np.linspace(zmin_camb,zmax_camb,numz_camb)
w_interp = [np.interp(zlist_camb,zlist,window[i]) for i in range(len(window))]
correlated_windows = [[w_interp[i]*w_interp[j] for i in range(len(w_interp))] for j in range(len(w_interp))]
noise = [1/nibar[i] for i in range(len(nibar))]

6


## Convert P(k) into P($\ell$)

In [6]:
Pkd = np.reshape(pk_dimless,(len(param_values),1,num_values,1,1,numz,660))
Da = np.reshape(d_a,(len(param_values),1,num_values,1,1,numz,1))
Hz = np.reshape(H_z,(len(param_values),1,num_values,1,1,numz,1))
Wij =  np.reshape(correlated_windows,(1,1,1,len(zavg),len(zavg),numz,1))

# m1 = Hz * Da
# print(np.shape(m1))
# m2 = m1 * Wij
# print(np.shape(m2))

# print(np.shape(m2[0]))

In [7]:
# del m1

# s1 = m2[0][0]*Pkd[0]
# save('../Axion-main/s1.npy',s1)
# print('done')

# del s1
# s2 = m2[1][0]*Pkd[0]
# save('../Axion-main/s2.npy',s2)
# print('done')

# del s2
# s3 = m2[2][0]*Pkd[0]
# save('../Axion-main/s3.npy',s3)
# print('done')

# del s3
# s4 = m2[3][0]*Pkd[0]
# save('../Axion-main/s4.npy',s4)
# print('done')

# del s4
# s5 = m2[4][0]*Pkd[0]
# save('../Axion-main/s5.npy',s5)
# del s5
# print('done')

In [8]:
# # Re-shape arrays
# Pkd = np.reshape(pk_dimless,(len(param_values),11,num_values,1,1,numz,660))
# Da = np.reshape(d_a,(len(param_values),1,num_values,1,1,numz,1))
# Hz = np.reshape(H_z,(len(param_values),1,num_values,1,1,numz,1))
# Wij =  np.reshape(correlated_windows,(1,1,1,len(zavg),len(zavg),numz,1))

# # Compute k-space integrand
# # m1 = Hz * Da
# # m2 = m1 * Wij
m = Hz * Da * Wij * Pkd
# print(np.shape(m))
# m = m2 * Pkd

# multipole values at which to compute P(ell)
lgoal = np.arange(2,3000,1)
# interpolate from k to ell
lmat = np.zeros((1,1,1,1,1,numz,2998))
for i in range(numz):
    lmat[0][0][0][0][0][i] = lgoal
kmat = lmat/Da

/tmp/ipykernel_2545/1305783667.py:20: RuntimeWarning: divide by zero encountered in true_divide
  kmat = lmat/Da


In [9]:
del Pkd
del pk
del Da
del Hz
del Wij

In [10]:
# import time
# m = Hz * Da * Wij * Pkd
# print(np.shape(m))
# integrand = []
# parameter dimension
for i in range(len(param_values)):
    print('Running parameter',i+1,'/ 5')
#     m = np.load('../Axion-main/s%s.npy'%(i+1))
#     print('loading done')
#     print(np.shape(m))
    integrand = []
    h1 = []
    # axion mass dimension
    for x in range(1):#(len(ma))
        h2 = []
        # parameter value dimension
        for y in range(num_values):
            h3 = []
            # zbin dimension 1
            for j in range(len(zavg)):
                h4 = []
                # zbin dimension 2 (for auto and cross-correlating zbins)
                for e in range(len(zavg)):
                    h5 = []
                    # redshift dimension
                    for f in range(numz):
                        # convert k to ell
                        spline = CubicSpline(k[i][x][y][f],m[0][x][y][j][e][f])
                        h5.append(spline(kmat[i][0][y][0][0][f]))
                        del spline
#                     m = np.delete(m,0,axis=3)
#                     print(np.shape(m))
                    h4.append(h5)
                    del h5
                h3.append(h4)
                del h4
#             for z in range(1):
#                 m = np.delete(m,0,axis=3)
#                 print(np.shape(m))
            h2.append(h3)
            del h3
#         for w in range(1):
#             m = np.delete(m,0,axis=2)
#             print(np.shape(m))
        h1.append(h2)
        del h2
#         save('../Axion-main/h1_%s_%s'%(i+1,x+1),h1)
#     for v in range(1):
#         m = np.delete(m,0,axis=1)
#         print(np.shape(m))
    m = np.delete(m,0,axis=0)
#     print(np.shape(m))
#     del m
    integrand.append(h1)
    del h1
    save('../Axion-main/integrand%s'%(i+1),integrand)
    del integrand
    print('step',i+1,'done')

del m
del k

Running parameter 1 / 5
step 1 done
Running parameter 2 / 5
step 2 done
Running parameter 3 / 5
step 3 done
Running parameter 4 / 5
step 4 done
Running parameter 5 / 5
step 5 done


In [11]:
del kmat
del d_a
del H_z
del a 
del param_array
del E 
del matter
del H0_i

In [ ]:
save('../Axion-main/integrand.npy',integrand)

In [ ]:
# zmin_camb = 0
# zmax_camb = 4.5
# numz_camb = 301
# zlist_camb = np.linspace(zmin_camb,zmax_camb,numz_camb)

z1 = np.zeros((60))
z2 = np.zeros((60))
z3 = np.zeros((60))
z4 = np.zeros((60))
z5 = np.zeros((61))

for i in range(301):
    if i<60:
        z1[i] = zlist_camb[i]
    elif 60<=i<120:
        z2[i-60] = zlist_camb[i]
    elif 120<=i<180:
        z3[i-120] = zlist_camb[i]
    elif 180<=i<240:
        z4[i-180] = zlist_camb[i]
    else:
        z5[i-240] = zlist_camb[i]

# print(z1,z2,z3,z4,z5)
# print(zlist_camb)

In [ ]:
# itg1 = integrand[0]
# # print(np.shape(itg1))
# save('../Axion-main/itg1.npy',itg1)
# print('done')

# del itg1
# itg2 = integrand[1]
# save('../Axion-main/itg2.npy',itg2)
# print('done')

# del itg2
# itg3 = integrand[2]
# save('../Axion-main/itg3.npy',itg3)
# print('done')

# del itg3
# itg4 = integrand[3]
# save('../Axion-main/itg4.npy',itg4)
# print('done')

# del itg4
# itg5 = integrand[4]
# save('../Axion-main/itg5.npy',itg5)
# del itg5
# print('done')

In [ ]:
# del integrand

In [ ]:
integrated_1 = np.trapz(np.nan_to_num(integrand),z1,dx = .015,axis = 5)
save('./Axion-main/integranted_1.npy',integrated_1)
del integranted_1

integrated_2 = np.trapz(np.nan_to_num(integrand),z2,dx = .015,axis = 5)
save('./Axion-main/integranted_2.npy',integrated_2)
del integranted_2

integrated_3 = np.trapz(np.nan_to_num(integrand),z3,dx = .015,axis = 5)
save('./Axion-main/integranted_3.npy',integrated_3)
del integranted_3

integrated_4 = np.trapz(np.nan_to_num(integrand),z4,dx = .015,axis = 5)
save('./Axion-main/integranted_4.npy',integrated_4)
del integranted_4

integrated_5 = np.trapz(np.nan_to_num(integrand),z5,dx = .015,axis = 5)
save('./Axion-main/integranted_5.npy',integrated_5)
del integranted_5

In [ ]:
# #If the memory runs out, use the Pipeline_integral notebook
# integrand1 = np.load('../Axion-main/itg1.npy')
# print(np.shape(integrand1))
# integrated_1 = np.trapz(np.nan_to_num(integrand1),zlist_camb,dx = .015,axis = 4)
# del integrand1
# integrated = integrated_1
# print('1/5 done')

# integrand2 = np.load('../Axion-main/itg2.npy')
# integrated_2 = np.trapz(np.nan_to_num(integrand2),zlist_camb,dx = .015,axis = 4)
# del integrand2
# print('2/5 done')

# integrand3 = np.load('../Axion-main/itg3.npy')
# integrated_3 = np.trapz(np.nan_to_num(integrand3),zlist_camb,dx = .015,axis = 4)
# del integrand3
# print('3/5 done')

# integrand4 = np.load('../Axion-main/itg4.npy')
# integrated_4 = np.trapz(np.nan_to_num(integrand4),zlist_camb,dx = .015,axis = 4)
# del integrand4
# print('4/5 done')

# integrand5 = np.load('../Axion-main/itg5.npy')
# integrated_5 = np.trapz(np.nan_to_num(integrand5),zlist_camb,dx = .015,axis = 4)
# del integrand5
# integrated = np.stack([integrated_1,integrated_2,integrated_3,integrated_4,integrated_5])
# del integrated_1
# del integrated_2
# del integrated_3
# del integrated_4
# del integrated_5
# print('5/5 done')

In [ ]:
# integrate over redshift
# integrated = np.trapz(np.nan_to_num(integrand),zlist_camb,dx = .015,axis = 5)
# P(ell) prefactor
prefactor = np.reshape((2*np.square(np.pi))/((2.998*(10**5))*np.power(lgoal,3)),(1,1,1,1,1,2998))
# angular power spectrum
pl = integrated*prefactor
del integrated
del prefactor
print('partly done')
add_noise = np.transpose(pl,(3,4,0,1,2,5))
shot_noise = np.reshape(noise,(len(zavg),1,1,1,1))
for i in range(len(zavg)):
    add_noise[i][i] += shot_noise[i]
Pl = np.transpose(add_noise,(2,3,4,0,1,5))
# np.save('../Axion-main/Pl_pipeline.npy',Pl)
print(np.shape(Pl))

In [ ]:
plt.loglog(k[2][0][0][0],pk_dimless[2][0][0][0],color = 'k',label = r'$\Lambda {\rm CDM}$')
plt.loglog(k[2][0][4][0],pk_dimless[2][0][4][0],color = 'r',label = r'$m_a = 10^{-22}{\rm eV}, f_a = 10\%$')
plt.loglog(k[2][0][4][0],pk_dimless[2][0][4][0],color = 'b',label = r'$m_a = 10^{-27}{\rm eV}, f_a = 10\%$')
plt.loglog(k[2][0][8][0],pk_dimless[2][0][8][0],color = 'g',label = r'$m_a = 10^{-27}{\rm eV}, f_a = 20\%$')
plt.loglog(k[2][0][0][0],pk_dimless[2][0][0][0],color = 'k')
plt.legend(frameon = False,loc = 'lower right',fontsize = 14)
plt.xlabel(r'$k (h/{\rm Mpc})$',size = 14)
plt.ylabel(r'$\Delta^2(k)$',size = 14)
plt.xlim(0.01,100)
plt.ylim(0.001,50)
plt.savefig('k_vs_Pk')
plt.show()
plt.close('all')

plt.plot(lgoal,Pl[0][0][0][0][0],color = 'cyan',label = zavg[0])
plt.plot(lgoal,Pl[0][0][0][1][1],color = 'navy',label = zavg[1])
plt.plot(lgoal,Pl[0][0][0][2][2]/2,color = 'lime',label = zavg[2])
plt.plot(lgoal,Pl[0][0][0][3][3]/4,color = 'darkgreen',label = zavg[3])
plt.plot(lgoal,Pl[0][0][0][4][4]/8,color = 'pink',label = zavg[4])
plt.plot(lgoal,Pl[0][0][0][5][5]/16,color = 'deeppink',label = zavg[5])
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$\ell$',size = 15)
plt.ylabel(r'$P(\ell)$',size = 15)
plt.legend()
plt.show()
plt.close('all')

for i in range(len(zavg)):
    plt.plot(zlist,window[i],color = 'deeppink')
plt.xlabel(r'$z$',size = 15)
plt.ylabel(r'$W_i(z)$',size = 15)
plt.show()
plt.close('all')

for i in range(len(zavg)):
    for j in range(len(zavg)):
        if i == j:
            plt.plot(zlist_camb,correlated_windows[i][j],color = 'b')
        else:
            plt.plot(zlist_camb,correlated_windows[i][j],color = 'r')
plt.xlabel(r'$z$',size = 15)
plt.ylabel(r'$W_i(z)W_j(z)$',size = 15)
plt.show()
plt.close('all')